# The Pupil Raw Features Extracted in Batch Pipeline.
## Introuduction
The features to be extracted includes blinking rate and chunks of wavelet coefficients.

## Reference
1. The previous codes.https://github.com/BaiYunpeng1949/MobileEyeComputing/tree/master/ProcessDataNTestAlgorithm
2. TODO: add papers here.
3. My work: https://docs.google.com/document/d/1oLv3oJQLjst1_pYgd_UA3RRL1fRGSbZ6uvlMuxmZR2k/edit#heading=h.r01ccf7ox05g

## Implementation

In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import plotly.express as px
from sklearn.metrics import r2_score
from scipy.special import logsumexp
from scipy.spatial.distance import euclidean
from sklearn.model_selection import TimeSeriesSplit
from fastdtw import fastdtw
from os import walk
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import datetime
import csv
import os, sys, warnings
import pywt, math
import argparse
import scipy
from numba import jit
from matplotlib import pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

In [2]:
def plotter(ax, data1, data2, param_dict):
    out = ax.plot(data1, data2, **param_dict)
    return out

## File-wise/Task-wise Data Pre-processing


### Constants Configurations

In [3]:
#######################################################################################
# Utilized in manipulating two eyes' data.
TS = 'Timestamp'
CF = 'Confidence'
DM = 'Diameter'
LDM = 'Left Diameter'
RDM = 'Right Diameter'
LCF = 'Left Confidence'
RCF = 'Right Confidence'
AVEDM = 'Average Diameter'
AVECF = 'Average Confidence'
DIFFDM = 'Difference Diameter'  # By my default: Left - Right - unit in pixels.
SR = 'Averaged Sampling Rate'
EVENT = 'Event'
TARGET_TASK = 'sitting'

# Interpolation.
INTERPOLATE_TYPE = 'linear'
LIMIT_DIRECTION = 'both'

#######################################################################################
# Used in the de-blinking part.
# Deblink.
BLINK_EXTENSION_TIMEWINDOW = 0.2 # 200ms
MIN_CF = 0.25
MIN_NUM_SAMPLE_BLINKS = 2

# Smooth.
WIN_TYPE = 'hann'
HANN_WINDOW_SIZE = 5

# Interpolate.
CURVE_TYPE = 'linear'
CURVE_ORDER = 3

# Column configuration.
ISBLINK = 'isBlink'
ISBLINK_LEFT = 'isBlink-Left'
ISBLINK_RIGHT = 'isBlink-Right'

AVE_DM = 'Averaged Diameter'
DIFF_DM = 'Difference Diameter'

# Used in the artifact rejection part.
HAMPLE_WIN_SIZE = 10

#######################################################################################
# Used in segmenting the time windows.
SAMPLING_RATE = 120 # The unit is Hertz.

# Used in the feature generating parts.
wavelet_decomposition_level = 2
wavelet = 'sym16'

IPA_LEFT = 'IPA Left'
IPA_RIGHT = 'IPA Right'

LHIPA_LEFT = 'LHIPA Left'
LHIPA_RIGHT = 'LHIPA Right'

MEAN_LEFT = 'Mean Left'
MEAN_RIGHT = 'Mean Right'

STD_LEFT = 'STD Left'
STD_RIGHT = 'STD Right'

SKEW_LEFT = 'Skew Left'
SKEW_RIGHT = 'Skew Right'

MAX_LEFT = 'MAX Left'
MAX_RIGHT = 'MAX Right'

MED_LEFT = 'Med Left'
MED_RIGHT = 'Med Right'

VAR_LEFT = 'Var Left'
VAR_RIGHT = 'Var Right'

# Experimental conditions
# Luminance.
LUX = 'Luminance'
# Task difficulty - labels.
LABELS = 'Labels'
PID = 'PID'

#######################################################################################
# Used in the run-in-batch part.
# File read configurations.
TWODMODE = '2D'
LEFT = 'left'
RIGHT = 'right'

# Experimental conditions
# Luminance.
LOW = 'lowlux'
MID = 'middlelux'
HIGH = 'highlux'
LUXS_SET = [LOW, MID, HIGH]
# Task difficulty - labels.
NOBACK = 'nothing'
ONEBACK = 'ONEBACK'
TWOBACK = 'TWOBACK'
THREEBACK = 'THREEBACK'
TASKDIFFS_SET = [NOBACK, ONEBACK, TWOBACK, THREEBACK]

#######################################################################################
# Configuration for Time-Series Cross Validation.
TRAIN_TEST_SPLIT = LABELS + ' Train and Test'
VALIDATION_SPLIT = LABELS + ' Validation'
TRAIN = 'train'
VALIDATION = 'val'
TEST = 'test'

### [Suspended] Left and Right Eye Synchronization and Calculate the Difference
Since I now implement ML/AI models to do a more inclusive analysis where more features could be analyzed, I will use the feature of the difference between left and right eye data.


1. Referenced from the previous work located as: https://github.com/BaiYunpeng1949/MobileEyeComputing/blob/master/ProcessDataNTestAlgorithm/LeftRightEyesSyncData.ipynb

2. The difference between two eyes were found correlated with cognitive workload estimation as well, see the related paper: Optimizing the usage of pupillary based indicators for cognitive workload. Reading link: https://docs.google.com/document/d/1jBezc9kqaziGlWk6sSgCvyjHTlpD5G6OoNZyh7K2HIo/edit#heading=h.qqfv1ot6zjd8

Besides, there is a considerable difference in pupil size variation for right and left eyes of the participants. See the paper: Exploring pupil size variation as a cognitive load indicator in visualization studies, link: https://drive.google.com/file/d/1z8O1NGNYA87La-CVMSr7cQkJ-VVOOUSe/view

In [4]:
# def _pre_dtw_sync(left_eye_file_path, right_eye_file_path):   
#     # Configure the parameters
#     entity_dwt_align = 'Timestamp'
#     entity_dwt_apply_dia = 'Diameter'
#     entity_dwt_apply_conf = 'Confidence'
#     left_eye = 'left'
#     right_eye = 'right'
    
#     # Tag labels
#     SAMPLING_RATE_LEFT = int((left_eye_file_path.split('_')[-1]).split('Hz')[0])
#     data_left = pd.read_csv(left_eye_file_path)

#     SAMPLING_RATE_RIGHT = int((right_eye_file_path.split('_')[-1]).split('Hz')[0])
#     data_right = pd.read_csv(right_eye_file_path)
    
#     df_left = data_left[['Timestamp','Confidence','Diameter','Event']].copy()
#     df_right = data_right[['Timestamp','Confidence','Diameter','Event']].copy()
    
#     # Determine the left and right eye data's size first: which one is bigger?
#     # Identify the number of elements in the left/right eyes data. If one applies the up-sampling method, the larger eye needs to be put in the first argument.
#     len_left = len(df_left)
#     len_right = len(df_right)
#     if len_left >= len_right:
#         df_reference = df_left.copy()  # df_reference: the one that being put in the first argument, as a reference.
#         df_alignment = df_right.copy() # df_alignment: the one that being put in the second argument, to be aligned to the reference.
#         df_origin = df_left.copy()
#         SR_SYNC = SAMPLING_RATE_LEFT
#     elif len_left < len_right:
#         df_reference = df_right.copy()
#         df_alignment = df_left.copy()
#         df_origin = df_right.copy()
#         SR_SYNC = SAMPLING_RATE_RIGHT
    
#     # Calculate for warping.
#     distance, path = fastdtw(df_reference[entity_dwt_align], df_alignment[entity_dwt_align], dist=euclidean)
    
#     return path, df_reference, df_alignment, entity_dwt_apply_dia, entity_dwt_apply_conf, df_origin, SR_SYNC

In [5]:
# # Define a function scnchronize and align/merge 2 eyes' numerical values, including diamter and confidence values.
# # The method of getting the average/mean value of 2 eyes' data as computing target is referenced from the mention in LHIPA.
# def _dtw_synchronize_merge_data(path_dwt, df_reference, df_alignment, entity_dwt_apply):
#     # Synchronize
#     data_sync = []
#     for i in range(0, len(path_dwt)):
#         data_sync.append([path_dwt[i][0],  # The index column is for dropping out duplicates.
#                          df_reference[entity_dwt_apply].iloc[path_dwt[i][0]],
#                          df_alignment[entity_dwt_apply].iloc[path_dwt[i][1]]])
#     df_sync = pd.DataFrame(data=data_sync,
#                            columns=['Index', 
#                                     'Reference '+entity_dwt_apply, 
#                                     'Alignment '+entity_dwt_apply]).dropna()
#     df_sync = df_sync.drop_duplicates(subset=['Index']) # Drop the duplicates according to the index of the reference.
#     df_sync = df_sync.reset_index(drop=True)
#     # Merge/Align
#     df_sync['Avg'+entity_dwt_apply] = df_sync.loc[:, ['Reference '+entity_dwt_apply, 'Alignment '+entity_dwt_apply]].mean(axis = 1)
#     # Calculate the difference
#     df_sync['Diff'+entity_dwt_apply] = df_sync['Reference '+entity_dwt_apply] - df_sync['Alignment '+entity_dwt_apply]
    
#     return df_sync

In [6]:
# # Synchronize the given 2 eyes' data.
# def dtw_sync(left_eye_file_path, right_eye_file_path):
#     # Prepare for the sychronization.
#     path, df_reference, df_alignment, entity_dwt_apply_dia, entity_dwt_apply_conf, df_origin, SR_SYNC = _pre_dtw_sync(left_eye_file_path = left_eye_file_path, 
#                                                                                                                   right_eye_file_path = right_eye_file_path)

    
#     # Synchronize, merge, and label data.
#     df_sync_dia = _dtw_synchronize_merge_data(path_dwt=path,
#                                               df_reference=df_reference,
#                                               df_alignment=df_alignment,
#                                               entity_dwt_apply=entity_dwt_apply_dia)
#     df_sync_conf = _dtw_synchronize_merge_data(path_dwt=path,
#                                                df_reference=df_reference,
#                                                df_alignment=df_alignment,
#                                                entity_dwt_apply=entity_dwt_apply_conf)
    
#     # Integrate into one dataframe.
#     df_sync = pd.DataFrame()
#     df_sync['Timestamp'] = df_origin['Timestamp']
#     df_sync['Confidence'] = df_sync_conf['AvgConfidence']
#     df_sync['Diameter'] = df_sync_dia['AvgDiameter']
#     df_sync['Event'] = df_origin['Event']
#     df_sync['DiffDiameter'] = df_sync_dia['DiffDiameter']
    
#     # Output and save into a csv file.
#     df_export = df_sync.copy()
#     file_name = left_eye_file_path.split('/')[-2:]

#     folder_path = '../Data/PreprocessedData/' + file_name[0] + '/'
#     if os.path.exists(folder_path) is False:
#         os.makedirs(folder_path)

#     write_file_name = 'synchronized_' + str(SR_SYNC) + 'Hz.csv'
#     write_file_path = folder_path + write_file_name
#     df_export.to_csv(write_file_path)
    
#     return df_sync

### Merge Two Eyes' Data using Timestamps, Interpolate, and Calculate
Reference: a blog introducing 2 sensors' data fusion - https://stackoverflow.com/questions/14079766/synchronize-dataset-multiple-users-multiple-timestamps

#### Implementation
To be noted that, when using the interpolation, we need to avoid overshooting, which is easily caused by spline curves. Otherwise we will have plenty negative pupil diameters and large outliers. I would use the 'linear' method to interpolate as suggested by Marshall. Apart from that, one might also want to try [monotonic](https://stackoverflow.com/questions/40072420/interpolate-without-having-negative-values-in-python) interpolators to avoid overshootings.

In [7]:
def upsample_timestamps_sync(left_eye_file_path, right_eye_file_path):
    # Read data from csv into dataframes.
    df_left = pd.read_csv(left_eye_file_path)
    df_right = pd.read_csv(right_eye_file_path)
    df_left = df_left[[TS,CF,DM,EVENT]].copy()
    df_right = df_right[[TS,CF,DM,EVENT]].copy()

    # Collect data from the targetted events: sitting.
    df_left = df_left.loc[df_left[EVENT] == TARGET_TASK]
    df_right = df_right.loc[df_right[EVENT] == TARGET_TASK]
#     print(len(df_left[df_left[EVENT]=='default']))
#     print(len(df_right[df_right[EVENT]=='default']))
    
    # Get the diameter data indexed by timestamps.
    left_diameters = df_left[DM].to_numpy() 
    series_left = pd.Series(left_diameters, index=df_left[TS])
    right_diameters = df_right[DM].to_numpy() 
    series_right = pd.Series(right_diameters, index=df_right[TS])
    
    # Synchronize 2 eyes' data by listing all timestamps, this process is actually a up-sampling.
    df_sync = pd.DataFrame([series_left, series_right]).T.sort_index()
    df_sync = df_sync.rename(columns={0: LDM, 1: RDM})

    #Interpolate all the NAN values using 'Spline 3' method with a bi-directional strategy to fill both the first and the last NAN values.
    df_sync = df_sync.interpolate(method=INTERPOLATE_TYPE, limit_direction=LIMIT_DIRECTION, axis=0)
#     df_sync = df_sync.ffill()
    
    # Align the confidence values according to the timestamps. 
    # Reference: Adding values in new column based on indexes with pandas in python. https://stackoverflow.com/questions/45636105/adding-values-in-new-column-based-on-indexes-with-pandas-in-python
    df_sync[LCF] = df_sync.index.to_series().map(df_left.set_index(TS)[CF])
    df_sync[RCF] = df_sync.index.to_series().map(df_right.set_index(TS)[CF])
    
    # Interpolate NAN values using the normal linear method.
    df_sync[LCF] = df_sync[LCF].interpolate(method=INTERPOLATE_TYPE, limit_direction=LIMIT_DIRECTION, axis=0)
    df_sync[RCF] = df_sync[RCF].interpolate(method=INTERPOLATE_TYPE, limit_direction=LIMIT_DIRECTION, axis=0)
    
#     # Get the difference and average of two eyes' diameter data and confidence values.
#     df_sync[AVEDM] = (df_sync[LDM] + df_sync[RDM]) / 2
#     df_sync[DIFFDM] = df_sync[LDM] - df_sync[RDM]
#     df_sync[AVECF] = (df_sync[LCF] + df_sync[RCF]) / 2
    
    # Get a new column storing the current trial's averaged sampling rate (the up-sampled version by interpolation).
    ave_samp_rate = len(df_sync) / (df_sync.index[-1] - df_sync.index[0])
    df_sync.loc[:, SR] = ave_samp_rate
    df_sync = df_sync.copy()
    
    return df_sync

### Deblinks and Blinking Rate Extraction

Reference: Check how David Linderbaure's group deal with the blinks. Use confidence to identify blinks. They removed the data within 200ms. However, they did not interpolate the eliminated ones. Here I clean data before and after 200ms of blinks. The input is the numpy data list of the "confidence" conlumn. Then return a list that marks which indecies are blinks. 

#### Implementation

In [8]:
def deblinks(df_input, confidence_column_label, diameter_column_label, isblink_column_label):
    # Feed in the dataframe to be processed and specific columns of the same eye/the averaged eye.
    # To be noted that the input was indexed by the timestamps. 
    # One Has to use the form of df_input[Col][df_input.index[i]] to reach the ith element.
    df = df_input.copy()
    # Initiate all 0 values to the IsBlink column.
    df.loc[:,isblink_column_label] = 0
    df = df.copy()
    
    # Parameter initilization
    blinks = []
    num_samples = len(df)
    i = 0 # The index starter.
    
    # Identify the blinks according to the low confidence values.
    while i < num_samples:
        if df[confidence_column_label][df.index[i]] < MIN_CF and i < num_samples -1:
            offset = 1
            next_data = df[confidence_column_label][df.index[i+offset]]
            while next_data < MIN_CF:
                offset = offset + 1
                if i + offset >= (num_samples - 1): # Check wheter exceeding the indecies boundary.
                    break
                next_data = df[confidence_column_label][df.index[i+offset]]
            
            # Judge whether the current index exceeds the 200ms time window.
            if offset >= MIN_NUM_SAMPLE_BLINKS:
                blinks.append((i, offset))
            
            i = i + offset
        else:
            i = i + 1
    
    # Mark data before and after BLINK_EXTENSION_TIMEWINDOW of samples.
    for j in range(len(blinks)):
        blink_index = blinks[j][0]
        blink_length = blinks[j][1]
        
        # Mark blinks within the searched area as np.nan values.
        for j in range(0, blink_length):
            df[diameter_column_label][df.index[blink_index + j]] = np.nan # Flag an NAN for the blinkings' diameters.
            df[isblink_column_label][df.index[blink_index + j]] = 1 # Flag a numerical value 1 for the blinkings.
        
        # Search for the time window with a length of 200ms. Then also mark blinks with np.nan values for the convenience of interplating.
        # Decremnenting.
        blink_start_timestamp = df.index[blink_index]
        k_dec = 0
        decrement_index = blink_index - k_dec
        # Controlled by the boundary conditions.
        while decrement_index >= 0:
            dec_timestamp = df.index[decrement_index]
            if blink_start_timestamp - dec_timestamp >= BLINK_EXTENSION_TIMEWINDOW:
                break
            else:
                df[diameter_column_label][df.index[decrement_index]] = np.nan # Set an NAN flag for data processing on the blinking data.
                df[isblink_column_label][df.index[decrement_index]] = 1 # Set an numerical flag on the blinking data.
                k_dec = k_dec + 1
                decrement_index = blink_index - k_dec
        
        # Incrementing - check the boundary limits first.
        blink_stop_timestamp = df.index[blink_index + blink_length]
        k_inc = 0
        increment_index = blink_index + blink_length + k_inc
        while increment_index < num_samples:
            inc_timestamp = df.index[increment_index]
            if inc_timestamp - blink_stop_timestamp >= BLINK_EXTENSION_TIMEWINDOW:
                break
            else:
                df[diameter_column_label][df.index[increment_index]] = np.nan # Set an NAN flag for data processing on the blinking data.
                df[isblink_column_label][df.index[increment_index]] = 1 # Set an numerical flag on the blinking data.
                k_inc = k_inc + 1
                increment_index = blink_index + blink_length + k_inc
    
    
    # Smooth the data - [Suspended] - Since the objective of freqeuncy-based analysis was to detect singularities, smooth should not be included here.
    # Besides, in Marshall's patent, https://patentimages.storage.googleapis.com/91/2f/5f/236d6711dcf6b6/US6090051.pdf, smooth was not applied.
#     df[diameter_column_label] = df[diameter_column_label].rolling(window=HANN_WINDOW_SIZE, center=True, win_type=WIN_TYPE).mean()
    
    # Interpolate the data - included in Marshall's patent - however the interpolation would introduce a lot of large negative values.
    df[diameter_column_label] = df[diameter_column_label].interpolate(method=CURVE_TYPE,order=CURVE_ORDER, limit_direction='both', axis=0)
    
    df_output = df.copy()
    return df_output

### Artefact Rejection

#### Implementation

In [9]:
## This part is directly cited from Sam's work.

# Filtering outliers 
# Lan et al. 2020 - median filter with sliding window of 10s
# Testing with numba optimised for-loop implementation of a Hampel Filter
# Note to self: I think this filter is also commonly used for pupil diameter filtering
@jit(nopython=True)
def hampel_filter_forloop_numba(input_series, window_size, n_sigmas=3):
    
    n = len(input_series)
    new_series = input_series.copy()
    k = 1.4826 # scale factor for Gaussian distribution
    indices = []
    
    for i in range((window_size),(n - window_size)):
        x0 = np.nanmedian(input_series[(i - window_size):(i + window_size)])
        S0 = k * np.nanmedian(np.abs(input_series[(i - window_size):(i + window_size)] - x0))
        if (np.abs(input_series[i] - x0) > n_sigmas * S0):
            new_series[i] = x0
            indices.append(i)
    
    return new_series, indices

In [10]:
def rej_artifact(df_input, diameter_column_label):
    df = df_input.copy()
    x_, outlier_x_ = hampel_filter_forloop_numba(df[diameter_column_label].to_numpy(), HAMPLE_WIN_SIZE)
    df[diameter_column_label] = x_.tolist()
    df_output = df.copy()
    return df_output

## Time-series Data Visualization
In this part, I visualize pupil data for different cognitive activities.

In [11]:
def plot_pupil_diameters(df_input, title_label):
    fig, ax = plt.subplots()
    df_input[LDM].plot(ax=ax)  # Plot the left diameter data.
    df_input[RDM].plot(ax=ax, title='Clean and Preprocessed Pupil Diameters (in pixels)\n' + title_label)  # Plot the right diameter data.
    ax.legend(['Left Eye', 'Right Eye'])

## Segmenting Time Windows

In this part, I will use overlapped sliding windows to extract wavelet-related features.

The reference publications include:
1. [WiStress: Contactless Stress Monitoring Using Wireless Signals, 2021](https://dl.acm.org/doi/pdf/10.1145/3478121).
2. [Feature extraction for robust physical activity recognition, 2017](https://hcis-journal.springeropen.com/articles/10.1186/s13673-017-0097-2).
3. [Feature Engineering on Time-Series Data for Human Activity Recognition](https://towardsdatascience.com/feature-engineering-on-time-series-data-transforming-signal-data-of-a-smartphone-accelerometer-for-72cbe34b8a60).
4. [Indexing Cognitive Workload Based on Pupillary Response under Luminance and Emotional Changes, 2013](https://dl-acm-org.libproxy1.nus.edu.sg/doi/pdf/10.1145/2449396.2449428).

### Implementation
The time window segmentation will be conducted file-wisely, i.e., each trial's data produces their own windows. Hence that to be aligned to the data pre-processing's file-wise fashion.

By slicing data with overlapped sliding windows, the new instances will be created. The former instances are sample points, now is conjoint sliding window waiting to be transformed into multiple features described by wavelet decomposition.

From here, the sample points are regarded as averaged ones.

In [12]:
# I constructed by own overlapped sliding window here.
def segment(args, df_input):
    df = df_input.copy()
    
    # Parameter initialization.
    num_samples = len(df)
    i=0 # Store the sliding window's starting point.
    offset_next = 0 # Store the next sliding window's starting index.
    windows = []
    
    window_length = args.time_window_length
    overlap_length = args.overlap_length
    window_num = int(SAMPLING_RATE * window_length)
    overlap_num = int(SAMPLING_RATE * overlap_length)
    step_num = int(window_num - overlap_num)
    
    # Determine the number of sample points according to the standard sampling rate, i.e., 120 Hz.
    indices = np.arange(num_samples)
    shape = (indices.size - window_num + 1, window_num)
    stride = indices.strides * 2
    view = np.lib.stride_tricks.as_strided(indices, strides = stride, shape = shape)[0::step_num]
    output_2D_list = view.copy()
    for i in range(len(output_2D_list)):
        windows.append((output_2D_list[i][0], output_2D_list[i][-1]))
    
#     # Determine the number of sample points according to given sampling rate according to the timestamps.
#     # I suspend this part, then use the standard data sampling part is for union data points and union feature expansion while using wavelet decomposition.
#     windows = []
#     while i < (num_samples - 1):
#         starting_timestamp = df.index[i]
#         offset = 1
#         stopping_timestamp = df.index[i+offset]
#         while stopping_timestamp - starting_timestamp < window_length:
#             # Check whether reaches the overlapping edge.
#             if stopping_timestamp - starting_timestamp <= (window_length - overlap_length):
#                 offset_next = offset
#                 # Then it should stop
            
#             offset = offset + 1
#             if i + offset >= (num_samples - 1): # Check wheter exceeding the indecies boundary.
#                 break
#             stopping_timestamp = df.index[i+offset]
            
#         # Store the starting index and offset index.
#         windows.append((i, i+offset))
        
#         # Update the starting index.
#         i = i + offset_next
        
#     # Iteratively check whether the last window is too short, if it is shorter thant the overlapped area, then discard it.
#     while True:
#         last_starting_index = df.index[windows[-1][0]]
#         last_ending_index = df.index[windows[-1][1]-1]
#         last_time_window_length = last_ending_index - last_starting_index
#         if last_time_window_length <= overlap_length:
#             del windows[-1]
#         else
#             break
    
    return windows

## Wavelet Coefficient Extraction

This part generates frequency features using wavelet analysis.

And generate new instances, which is composed of several sample points from the time window.

### Wavelet-based Auxiliary Functions

In [13]:
# Find the maximum modulus.
def modmax(d):
    # Compute signal modulus.
    m = [0.0] * len(d)
    for i in range(len(d)):
        m[i] = math.fabs(d[i])

    # If value is larger than both neighbours , and strictly larger than either , then it is a local maximum.
    t = [0.0] * len(d)

    for i in range(len(d)):
        ll = m[i - 1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i + 1] if i < len(d) - 2 else m[i]

        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            # compute magnitude
            t[i] = math.sqrt(d[i] ** 2)
        else:
            t[i] = 0.0
    return t


# Calculate IPA
def calc_ipa(d, wavelet, duration):
    # Initialize parameter.
    wavelet = wavelet
    
    # obtain 2-level DWT of pupil diameter signal d
    try:
        (cA2,cD2,cD1) = pywt.wavedec(d, 'db8','per',level=2)
    except ValueError:
        return

    # get signal duration (in seconds)
    tt = duration

    # using data from Pedrotti et al
    # tt = 1.0

    # normalize by 1=2j , j = 2 for 2-level DWT
    cA2[:] = [x / math.sqrt(4.0) for x in cA2]
    cD1[:] = [x / math.sqrt(2.0) for x in cD1]
    cD2[:] = [x / math.sqrt(4.0) for x in cD2]

    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)

    # threshold using universal threshold lambda_univ = s*sqrt(p(2 log n))
    lambda_univ = np.std(cD2m) * math.sqrt(2.0 * np.log2(len(cD2m)))
    # where s is the standard deviation of the noise
    cD2t = pywt.threshold(cD2m, lambda_univ, mode="hard")

    # compute IPA
    ctr = 0
    for i in range(len(cD2t)):
        # print(cD2t[i])
        if math.fabs(cD2t[i]) > 0:
            ctr += 1

    # print(ctr)
    IPA = float(ctr) / tt
    return IPA 


# Calculate L/H IPA.
def calc_lhipa(d, wavelet, duration):
    """
    Here d is a list.
    """
    # Initialize parameter.
    wavelet = wavelet
    
    # Find max decomposition level.
    w = pywt.Wavelet(wavelet)
    maxlevel = pywt.dwt_max_level(len(d), filter_len=w.dec_len)

    # Set high and low frequency band indeces.
    hif, lof = 1, int(maxlevel / 2)

    # Get detail coefficients of pupil diameter signal d.
    cD_H = pywt.downcoef('d', d, wavelet, 'per', level=hif)
    cD_L = pywt.downcoef('d', d, wavelet, 'per', level=lof)

    # Normalize by 1/ 2j.
    cD_H[:] = [x / math.sqrt(2 ** hif) for x in cD_H]
    cD_L[:] = [x / math.sqrt(2 ** lof) for x in cD_L]

    # Obtain the LH:HF ratio.
    cD_LH = cD_L
    for i in range(len(cD_L)):
        # I tried to solve the RuntimeWarning nan issue by referring to a stackoverflow blog here https://stackoverflow.com/questions/27784528/numpy-division-with-runtimewarning-invalid-value-encountered-in-double-scalars
        if cD_L[i]== 0:
            cD_LH[i] = 0 # Sometimes, cD_L[i] would be 0 and cD_H[int((2 ** lof) / (2 ** hif)) * i] would be nan, and RuntimeWarning would be raised.
        else:
            cD_LH[i] = cD_L[i] / cD_H[int((2 ** lof) / (2 ** hif)) * i]   # Used a '//' instead of '/' to make sure the index is an integer. This line is the original line.

    # Detect modulus maxima , see Duchowski et al. [15].
    cD_LHm = modmax(cD_LH)

    # Threshold using universal threshold λuniv = σˆ (2logn), where σˆ is the standard deviation of the noise.
    lambda_univ = np.std(cD_LHm) * math.sqrt(2.0*np.log2(len(cD_LHm)))
    cD_LHt = pywt.threshold(cD_LHm, lambda_univ, mode="less")

    # Get signal duration (in seconds).
    # tt = d[-1].timestamp() - d[0].timestamp()
    tt = duration  # The unit was the second.

    # Compute LHIPA.
    ctr = 0
    for i in range(len(cD_LHt)):
        if math.fabs(cD_LHt[i]) > 0:
            ctr += 1
    LHIPA = float(ctr) / tt
    return LHIPA

### Implementation

In [14]:
def freq_analysis(args, df_input, windows_indices, j=wavelet_decomposition_level):
    df = df_input.copy()
    windows = windows_indices

    # Parameter initialization.
    freq_features_two_eyes = []
    freq_features_left = []
    freq_features_right = []
    column_name_left = 'Left-'
    column_name_right = 'Right-'
    
    ipas_left = []
    ipas_right = []
    lhipas_left = []
    lhipas_right = []
    
    means_left = []
    means_right = []
    stds_left = []
    stds_right = []
    medians_left = []
    medians_right = []
    maxs_left = []
    maxs_right = []
    skews_left = []
    skews_right = []
    variances_left = []
    variances_right = []
    
    blinking_rates_left = []
    blinking_rates_right = []
    aves= []
    diffs = []
    luxes = []
    labels = []
    pids = []
    
    for i in range(len(windows)):
        starting_index = windows[i][0]
        ending_index = windows[i][1] + 1
        
        # Get the left and right eyes' pupil diameters.
        data_left = df[LDM].to_list()[starting_index:ending_index]
        data_right = df[RDM].to_list()[starting_index:ending_index]
        
        # Get the frequency features.
        (cA2_left, cD2, cD1) = pywt.wavedec(data_left, wavelet, 'per', level=j)
        (cA2_right, cD2, cD1) = pywt.wavedec(data_right, wavelet, 'per', level=j)
        
        cA2_two_eyes = np.concatenate((cA2_left, cA2_right), axis=0)
        
        freq_features_left.append(cA2_left)
        freq_features_right.append(cA2_right)
        freq_features_two_eyes.append(cA2_two_eyes)
        
        # Get the IPA features for both eyes.
        ipa_left = calc_ipa(d=data_left, wavelet=wavelet, duration=args.time_window_length)
        ipa_right = calc_ipa(d=data_right, wavelet=wavelet, duration=args.time_window_length)
        ipas_left.append(ipa_left)
        ipas_right.append(ipa_right)
        
        # Get the LHIPA features for both eyes.
        lhipa_left = calc_lhipa(d=data_left, wavelet=wavelet, duration=args.time_window_length)
        lhipa_right = calc_lhipa(d=data_right, wavelet=wavelet, duration=args.time_window_length)
        lhipas_left.append(lhipa_left)
        lhipas_right.append(lhipa_right)
        
        # Get the averaged diameter feature and difference diameter features.
        left_ave = sum(data_left) / len(data_left)
        right_ave = sum(data_right) / len(data_right)
        ave = (left_ave + right_ave) / 2
        diff = left_ave - right_ave
        aves.append(ave)
        diffs.append(diff)
        
        # Get the pupil diameters statistical features. 
        data_left_np = np.array(data_left)
        data_right_np = np.array(data_right)
        # Reference: Eye Tracking-Based Stress Classification of Athletes in Virtual Reality, 2022.
        # Features: mean, std, skewness, max, median, variance.
        
        mean_left = np.mean(data_left_np)
        mean_right = np.mean(data_right_np)
        means_left.append(mean_left)
        means_right.append(mean_right)
        
        std_left = np.std(data_left_np)
        std_right = np.std(data_right_np)
        stds_left.append(std_left)
        stds_right.append(std_right)
        
        med_left = np.median(data_left_np)
        med_right = np.median(data_right_np)
        medians_left.append(med_left)
        medians_right.append(med_right)
        
        max_left = np.max(data_left_np)
        max_right = np.max(data_right_np)
        maxs_left.append(max_left)
        maxs_right.append(max_right)
        
        skew_left = scipy.stats.skew(data_left_np, axis=0, bias=True)
        skew_right = scipy.stats.skew(data_right_np, axis=0, bias=True)
        skews_left.append(skew_left)
        skews_right.append(skew_right)
        
        var_left = np.var(data_left_np, axis=0)
        var_right = np.var(data_right_np, axis=0)
        variances_left.append(var_left)
        variances_right.append(var_right)
        
        # Get the blinking rate feature from both eyes. df[df['col'] == value
        blinks_left = np.array(df[ISBLINK_LEFT].to_list()[starting_index:ending_index])
        blinking_rate_left = (np.sum(blinks_left))/len(blinks_left)
        blinking_rates_left.append(blinking_rate_left)
        
        blinks_right = np.array(df[ISBLINK_RIGHT].to_list()[starting_index:ending_index])
        blinking_rate_right = (np.sum(blinks_right))/len(blinks_right)
        blinking_rates_right.append(blinking_rate_right)
        
        # Get the luminance feature. df['City'].iat[0]
        luxes.append(df[LUX].iat[0])
        
        # Get the PID.
        pids.append(df[PID].iat[0])
        
        # Get the task label.
        labels.append(df[LABELS].iat[0])
        
    # Add high dimension features into the dataframe. Set the columns
    # From now, the instances are vertically conpacted by the sliding windows, but horizontally expaned.
    # Frequency features.
    horizontal_length_left = np.array(freq_features_left).shape[1]
    horizontal_length_right = np.array(freq_features_right).shape[1]
    left_features = []
    for i in range(horizontal_length_left):
        feature_name = column_name_left + str(i)
        left_features.append(feature_name)
        
    right_features = []
    for i in range(horizontal_length_right):
        feature_name = column_name_right + str(i)
        right_features.append(feature_name)
    
    feature_names = left_features + right_features
    df_freq = pd.DataFrame(freq_features_two_eyes, columns=feature_names)
    
    # IPA and LHIPA features for both eyes.
    df_freq[IPA_LEFT] = ipas_left
    df_freq[IPA_RIGHT] = ipas_right
    df_freq[LHIPA_LEFT] = lhipas_left
    df_freq[LHIPA_RIGHT] = lhipas_right
    
    # Diameter average and difference features.
    df_freq[AVE_DM] = aves
    df_freq[DIFF_DM] = diffs
    
    # Left and right eye diameters' statistical features.
    df_freq[MEAN_LEFT] = means_left
    df_freq[MEAN_RIGHT] = mean_right
    
    df_freq[MAX_LEFT] = maxs_left
    df_freq[MAX_RIGHT] = maxs_right
    
    df_freq[STD_LEFT] = stds_left
    df_freq[STD_RIGHT] = stds_right
    
    df_freq[SKEW_LEFT] = skews_left
    df_freq[SKEW_RIGHT] = skews_right
    
    df_freq[MED_LEFT] = medians_left
    df_freq[MED_RIGHT] = medians_right
    
    df_freq[VAR_LEFT] = variances_left
    df_freq[VAR_RIGHT] = variances_right
               
    # Blinking rate features.
    df_freq[ISBLINK_LEFT] = blinking_rates_left
    df_freq[ISBLINK_RIGHT] = blinking_rates_right
    
    # Luminance features.
    df_freq[LUX] = luxes
    
    # PID.
    df_freq[PID] = pids
    
    # Label.
    df_freq[LABELS] = labels
        
    df_output = df_freq.copy()
    return df_output

## Split the Dataset: Train, Validation, and Test

In this part I split the dataset into training part, validation part, and testing parts. To get ready for deep learning related methods where validations are necessary for updating parameters.

Since our model could be classified into 'Activity Recognition' and 'Time-series Forecasting Models', I followed their ways to split dataset to aviod dependancy problems / look-ahead bias. We should alwasy respect the temporal order in the dataset.Here I chose to use the 'walk forward validation' which is one of the most popular methods. The other possible solutions are 'Rep-Holdout' and 'hv holdout'.

### Reference
1. [Multiple time series forecasting: How to split the data for training of a neural network [duplicate]](https://stats.stackexchange.com/questions/564867/multiple-time-series-forecasting-how-to-split-the-data-for-training-of-a-neural)
2. [Using k-fold cross-validation for time-series model selection.](https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection)
3. [Splitting Time Series Data into Train/Test/Validation Sets.](https://stats.stackexchange.com/questions/346907/splitting-time-series-data-into-train-test-validation-sets)
4. [Stackoverflow: how to implement walk forward testing in sklearn?](https://stackoverflow.com/questions/31947183/how-to-implement-walk-forward-testing-in-sklearn)
5. [sklearn.model_selection.TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)

### Implementation

In [15]:
# This function label data with 'train', 'val', or 'test' to indicate their belonging groups.
def time_series_cross_validation(args, df_input):
    df = df_input.copy()
    # Split the test dataset first: get the most recent data.
    num_splits_test = (int(1 / args.test_pct) - 1) # To be noted that the split is 1 less than the partitions.
    tscv_test = TimeSeriesSplit(n_splits=num_splits_test)
    original_indecies = np.array(df.index)
    # Since the labels are attached with the features in the 'Labels' column, I don't need to split the labels specifically.
    counter = 0
    for train_indecies, test_indecies in tscv_test.split(X=df):
        counter = counter + 1
        # Only get the last indecies set for testing.
        if counter == num_splits_test:
            df.loc[train_indecies,TRAIN_TEST_SPLIT] = TRAIN
            df.loc[test_indecies,TRAIN_TEST_SPLIT] = TEST
            
            train_set_indecies = train_indecies
            test_set_indecies = test_indecies
    
    # Split the validation dataset in the remaining part.
    num_splits_val = int((1 - args.test_pct) / args.val_pct) - 1
    tscv_val = TimeSeriesSplit(n_splits=num_splits_val)
    
    index_validation = 0
    for train_indecies, val_indecies in tscv_val.split(X=train_set_indecies):
        # Spliting and labeling.
        current_val_labels = VALIDATION_SPLIT + '-' + str(index_validation)
        df.loc[train_indecies,current_val_labels] = TRAIN
        df.loc[val_indecies,current_val_labels] = VALIDATION
        df.loc[test_set_indecies,current_val_labels] = TEST
        
        # Update the index for labeling. Starting from 0 is expected easier for the later accessing data for training.
        index_validation = index_validation + 1
    
    # Output the results.
    df_output = df.copy()
    return df_output

## Run in Batch

This part I extract raw data and extract features in batch from all collected data.

### Auxiliary Functions

In [16]:
# My dumb verion of finding a list's member exisiting in a string or not, an alternative could be: https://thispointer.com/combine-for-loop-if-else-statement-in-python/
def check_string(input_string, target_lists):
    for x in target_lists:
        print(x)
        if x in input_string:
            output_string = x
            break
    return output_string

In [17]:
# Create a new folder to store this batch of calculation results.
def output_csv(args, df_input):
    dirpath_results = args.results_save_path
    now = datetime.datetime.now()
    datestamp = now.strftime("%d-%m-%H-%M")
    results_folder_path = dirpath_results + datestamp + '/'

    if os.path.exists(results_folder_path) is False:
        os.makedirs(results_folder_path)

    # Write the current dataframe as a csv into the new created folder.
    df_input.to_csv(results_folder_path + args.results_csv_filename, encoding='utf-8', index=False, header=True)  

### Run

In [18]:
# This part reads data file-wisely and generate designed features, and run the pipeline.
def run(args):
    # Initialize dataframes and storing lists.
    FEATURES_SET = [LUX, LABELS, PID]
    df_pre_features = pd.DataFrame(columns=FEATURES_SET)
    frames_pre_features = []
    
    # Start to read raw data from files.
    raw_data_path = args.path
    # List all directory names
    dirs_list = []
    for (dir_path, dir_names, file_names) in walk(raw_data_path):
        dirs_list.extend(dir_names)
        break
    
    # Traverse all the file names in a given directory.
    for dir_name in dirs_list:
        dir_path = raw_data_path + dir_name + '/'
        file_names_list = []
        df_get_labels = pd.DataFrame()
        df_current_trial = pd.DataFrame(columns=FEATURES_SET)

        for (_, _, file_names) in walk(dir_path):
            file_names_list.extend(file_names)

        # Find the targetted files.
        for file_name in file_names:
            if TWODMODE in file_name:
                if LEFT in file_name:
                    file_path_left = dir_path + file_name
                elif RIGHT in file_name:
                    file_path_right = dir_path + file_name

        # Step 1: Synchronize 2 eyes' targetted data (both remove the 'default' ones).
        df_sync = upsample_timestamps_sync(left_eye_file_path=file_path_left, right_eye_file_path=file_path_right)

        # Step 2: Deblink, and interpolate.
        # Process the left eye data.
        df_deblink = deblinks(df_input=df_sync, confidence_column_label=LCF, diameter_column_label=LDM, isblink_column_label=ISBLINK_LEFT)
        # Process the right eye data.
        df_deblink = deblinks(df_input=df_deblink, confidence_column_label=RCF, diameter_column_label=RDM, isblink_column_label=ISBLINK_RIGHT)

        # Step 3: Artifect rejection
        df_rej = rej_artifact(df_input=df_deblink, diameter_column_label=LDM)
        # Process the right eye data.
        df_rej = rej_artifact(df_input=df_rej, diameter_column_label=RDM)

        # Step 4: Segment data using an overlapping time window.
        windows = segment(args=args, df_input=df_rej)

        # Step 5: Get labels and experiment conditions.
        # Assign features into the current trial's dataframe.
        df_get_labels = df_rej.copy()
        # Lux conditions.
        lux_condition = [iterator for iterator in LUXS_SET if iterator in dir_name][0] # My learnt version from: https://thispointer.com/combine-for-loop-if-else-statement-in-python/
        df_get_labels.loc[:, LUX] = lux_condition
        df_get_labels = df_get_labels.copy()
        # Task difficulty labels.
        task_diff_condition = [iterator for iterator in TASKDIFFS_SET if iterator in dir_name][0]
        df_get_labels.loc[:, LABEL] = task_diff_condition
        df_get_labels = df_get_labels.copy()
        # PID.
        pid = dir_name.split('-')[-1]
        df_get_labels.loc[:, PID] = pid
        df_get_labels = df_get_labels.copy()
        
        # Step 6: Generate wavelet features.And re-set instances: vertically decrease but horizontally increase.
        df_freq_analyzed = freq_analysis(args=args, df_input=df_get_labels, windows_indices=windows)
        
        # Step 7: Split the dataset into traing, validation, and testing datasets.
        df_split = time_series_cross_validation(args=args, df_input=df_freq_analyzed)
    
    #     # Debug Area: Plot to pre-view pupil data. TODO: delete later, for debugging and preprocessing validation: is there any problems?
#         break
    #     plot_pupil_diameters(df_input=df_sync, title_label=dir_name + ' synchronized')
    #     plot_pupil_diameters(df_input=df_deblink, title_label=dir_name + ' delinked')
    #     plot_pupil_diameters(df_input=df_rej, title_label=dir_name + ' rejected artifact')
        
        # Wrap up.
        df_current_trial = df_split.copy()
        frames_pre_features.append(df_current_trial)

    # Merge dataframes/single trials' data into a large dataset.
    df_pre_features = pd.concat(frames_pre_features)
    
    # Output the results.
    df_output = df_pre_features.copy()
    return df_output

### Implementation - My Console

In [19]:
# Argument allocations.
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-f')
    parser.add_argument("--path", type=str, default='../Data/RawData4ML/VersionNovAll/', 
                        help="The path to read raw pupil diameter data.")
    parser.add_argument('--using_cuda', type=bool, default=True,
                        help='cuda/cpu')
    parser.add_argument('--gpu_ids', type=bool, default=[0],
                        help='cuda/cpu')
    parser.add_argument('--argument_save_path', type=str, default='../Data/Results/',
                        help='The path to save arguments.')
    parser.add_argument('--results_save_path', type=str, default='../Data/Results/',
                    help='The path to save processed data and generated features.')
    parser.add_argument('--results_csv_filename', type=str, default='results.csv',
                    help='The result csv filename.')
    parser.add_argument('--time_window_length', type=float, default=4.0,
                        help='The length of time window unitted in second.')
    parser.add_argument('--overlap_length', type=float, default=3.0,
                        help='The length of overlapped time window unitted in second.')
    parser.add_argument('--test_pct', type=float, default=0.1,
                        help='The percentage of the testing set of the whole dataset.')
    parser.add_argument('--val_pct', type=float, default=0.1,
                        help='The percentage of the validation set of the whole dataset.') # Hence that the traing:validation:testing = (1-test_pct-val_pct):val_pct:test_pct.
    return parser.parse_args(args = [])

In [20]:
# Implement.
# Get arguments.
args = parse_args()
df_features = run(args=args)
output_csv(args=args, df_input=df_features)